<a href="https://colab.research.google.com/github/tirtthshah/text-to-image-pipeline/blob/main/Task_3(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os

os.makedirs("/root/.kaggle", exist_ok=True)
os.rename("/content/kaggle.json", "/root/.kaggle/kaggle.json")

os.chmod("/root/.kaggle/kaggle.json", 0o600)

In [ ]:
import os

if os.path.exists('/content/kaggle.json'):
  print("kaggle.json found in /content")
else:
  print("kaggle.json not found in /content")

In [ ]:
!pip install -q kaggle
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset -p /content

In [ ]:
import zipfile

with zipfile.ZipFile("/content/brain-tumor-mri-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/brain_tumor_mri")

print(" Dataset extracted to /content/brain_tumor_mri")

In [ ]:
import os


image_dir = "/content/brain_tumor_mri"
for root, dirs, files in os.walk(image_dir):
    for file in files[:10]:
        print(os.path.join(root, file))

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

sample_path = os.path.join(image_dir, "Testing/meningioma/Te-me_0016.jpg")
img = Image.open(sample_path)

plt.imshow(img)
plt.axis("off")
plt.title("Sample MRI Image")
plt.show()

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [ ]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_path = "/content/brain_tumor_mri/Training"
test_path = "/content/brain_tumor_mri/Testing"

train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
class TumorClassifier(nn.Module):
    def __init__(self):
        super(TumorClassifier, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128), nn.ReLU(),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

In [ ]:
!pip install -q tqdm

In [ ]:
from tqdm import tqdm

for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loop.set_postfix(loss=loss.item())

    epoch_acc = 100 * correct / total
    print(f" Epoch {epoch+1} — Loss: {running_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import torch
import random

classes = train_dataset.classes
class_to_idx = train_dataset.class_to_idx

def imshow(img, label, pred):
    img = img.permute(1, 2, 0).numpy()
    plt.imshow(img)
    plt.title(f"Prediction: {classes[pred]}, Reallity: {classes[label]}")
    plt.axis("off")
    plt.show()

for chosen_class in classes:
    print(f"\n Class: {chosen_class}")
    target_idx = class_to_idx[chosen_class]

    filtered = [(img, label) for img, label in test_dataset if label == target_idx]
    if not filtered:
        print(" No images found for this class.")
        continue

    img, label = random.choice(filtered)
    img_tensor = img.unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        _, pred = torch.max(output, 1)

    imshow(img, label, pred.item())

Final Output( Prediction )

In [ ]:
from google.colab import files
from PIL import Image
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()])


uploaded = files.upload()
for filename in uploaded.keys():

    img = Image.open(filename).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        _, pred = torch.max(output, 1)


    plt.imshow(img)
    plt.title(f"Predicted Tumor Type: {train_dataset.classes[pred.item()]}")
    plt.axis("off")
    plt.show()